# SIGCUP 23

In [1]:
import os
from glob import glob
import numpy as np
import pandas as pd

import rasterio
from rasterio.features import shapes as RSHAPES
import cv2
import geopandas as gpd
import shapely

import tensorflow as tf

In [3]:
print('numpy version = ', np.__version__)
print('pandas version = ', pd.__version__)
print('rasterio version = ', rasterio.__version__)
print('cv2 version = ', cv2.__version__)
print('geopandas version = ', gpd.__version__)
print('shapely version = ', shapely.__version__)
print('tensorflow version = ', tf.__version__)

numpy version =  1.23.3
pandas version =  1.4.4
rasterio version =  1.3.8
cv2 version =  4.6.0
geopandas version =  0.14.0
shapely version =  2.0.1
tensorflow version =  2.10.0


In [6]:
# ------------- Please modify here ------------- 

# the directory of images to be evaluated 
DIR_INPUT_IMG = 'put here the path of image to be evaluted'
# DIR_INPUT_IMG = os.path.join('TrainData', 'Processed') 

ls_img_path = glob(os.path.join(DIR_INPUT_IMG, '*.tif'))
print(ls_img_path)
if len(ls_img_path) == 0: 
  print('Do not find the satellite images!')

# the output directory of geopackge
DIR_OUTPUT = 'GPKG'
if not os.path.exists(DIR_OUTPUT): 
  os.makedirs(DIR_OUTPUT)

['TrainData\\Processed\\2019-06-03_roi0.tif', 'TrainData\\Processed\\2019-06-03_roi1.tif', 'TrainData\\Processed\\2019-06-03_roi2.tif', 'TrainData\\Processed\\2019-06-03_roi3.tif', 'TrainData\\Processed\\2019-06-03_roi4.tif', 'TrainData\\Processed\\2019-06-03_roi5.tif', 'TrainData\\Processed\\2019-06-19_roi0.tif', 'TrainData\\Processed\\2019-06-19_roi1.tif', 'TrainData\\Processed\\2019-06-19_roi2.tif', 'TrainData\\Processed\\2019-06-19_roi3.tif', 'TrainData\\Processed\\2019-06-19_roi4.tif', 'TrainData\\Processed\\2019-06-19_roi5.tif', 'TrainData\\Processed\\2019-07-31_roi0.tif', 'TrainData\\Processed\\2019-07-31_roi1.tif', 'TrainData\\Processed\\2019-07-31_roi2.tif', 'TrainData\\Processed\\2019-07-31_roi3.tif', 'TrainData\\Processed\\2019-07-31_roi4.tif', 'TrainData\\Processed\\2019-07-31_roi5.tif', 'TrainData\\Processed\\2019-08-25_roi0.tif', 'TrainData\\Processed\\2019-08-25_roi1.tif', 'TrainData\\Processed\\2019-08-25_roi2.tif', 'TrainData\\Processed\\2019-08-25_roi3.tif', 'TrainDat

In [4]:
# ------------- Global Variable ------------- 
# DO NOT CHANGE 

IMG_WIDTH = 256
IMG_HEIGHT = 256

INPUT_X_CHANNEL = 3
INPUT_Y_CHANNEL = 2
OUTPUT_CHANNEL = 2

DIR_MODEL = 'model_ckp'


# Train


In [7]:
# functions 
def downsampling(filters, size):
  res = tf.keras.Sequential()
  res.add(tf.keras.layers.Conv2D(filters, size, strides=1, padding='same', use_bias=True))
  res.add(tf.keras.layers.BatchNormalization())
  res.add(tf.keras.layers.LeakyReLU())
  res.add(tf.keras.layers.Conv2D(filters, size, strides=2, padding='same', use_bias=True))
  res.add(tf.keras.layers.BatchNormalization())
  res.add(tf.keras.layers.LeakyReLU())
  return res


def upsampling(filters, size):
  res = tf.keras.Sequential()
  res.add(tf.keras.layers.Conv2DTranspose(filters, size, strides=2,padding='same', use_bias=True))
  res.add(tf.keras.layers.BatchNormalization())
  res.add(tf.keras.layers.LeakyReLU())
  res.add(tf.keras.layers.Conv2D(filters, size, strides=1, padding='same', use_bias=True))
  res.add(tf.keras.layers.BatchNormalization())
  res.add(tf.keras.layers.LeakyReLU())
  return res


def unet():
  down_stack = [
    downsampling(16, 3),  
    downsampling(32, 3), 
    downsampling(64, 3),
  ]

  up_stack = [
    upsampling(32, 3),  
    upsampling(16, 3), 
  ]

  x_trans = upsampling(INPUT_X_CHANNEL, 3)
  
  m_out = tf.keras.Sequential()
  m_out.add(tf.keras.layers.Conv2D(32, 1, strides=1, padding='same', activation=tf.nn.relu))
  m_out.add(tf.keras.layers.Conv2D(32, 1, strides=1, padding='same', activation=tf.nn.relu))
  m_out.add(tf.keras.layers.Conv2D(OUTPUT_CHANNEL, 1, strides=1, padding='same'))


  
  inputs = tf.keras.layers.Input(shape=[IMG_HEIGHT, IMG_WIDTH, INPUT_X_CHANNEL])
  x = inputs


  # UNet Flow
  skips = []

  for down in down_stack:
    x = down(x)
    skips.append(x)

  skips = reversed(skips[:-1])

  for up, skip in zip(up_stack, skips):
    x = up(x)
    x = tf.keras.layers.Concatenate()([x, skip])

  x = x_trans(x)

  x = tf.keras.layers.Concatenate()([x, inputs])

  output = m_out(x)
  
  return tf.keras.Model(inputs=inputs, outputs=output)


def moderator(): 
  down_stack = [
    downsampling(16, 3),  # (bs, 64, 64, 16)
    downsampling(32, 3),  # (bs, 32, 32, 32)
    downsampling(64, 3),  # (bs, 16, 16, 64)
  ]

  inputs = tf.keras.layers.Input(shape=[IMG_HEIGHT, IMG_WIDTH, INPUT_X_CHANNEL]) #RGB
  ls_logits = []
  for i in range(9):
    ls_logits.append(tf.keras.layers.Input(shape=[IMG_HEIGHT, IMG_WIDTH, OUTPUT_CHANNEL])) # logits from 9 branches
  model_input = [inputs] + ls_logits
  
  x = inputs
  
  for down in down_stack: 
    x = down(x)
    
  x = tf.reduce_mean(x, [0, 1, 2], keepdims=True) # [batch, 1, 1, 64]
  weights = tf.keras.layers.Conv2D(
    9, 1, strides=1,
    padding='same', 
    activation='softmax')(x)

  output = weights[...,0:1]*ls_logits[0]+weights[...,0:1] * ls_logits[0]
  for i in range(1, 9): 
    output += weights[..., i:i+1] * ls_logits[i]
    
  return tf.keras.Model(inputs=model_input, outputs=output)
 
 


In [8]:
# init models
ls_pretrain = []
ls_pretrain_opmz = []
ls_pretrain_ckp = []
for i in range(9):
  ls_pretrain.append(unet())
  ls_pretrain_opmz.append(tf.keras.optimizers.Adam(1e-4))
  ls_pretrain_ckp.append(tf.train.Checkpoint(u_optimizer=ls_pretrain_opmz[-1], u_model=ls_pretrain[-1]))
  
  dir_ckp_pretrained = os.path.join(DIR_MODEL, 'UNet4')
  if i > 0:
    dir_ckp_pretrained += '_sub'+str(i)
  with open(os.path.join(dir_ckp_pretrained, 'ckp', 'best.txt'), 'r') as f:
    best_ckp_path = f.readline()  
  ls_pretrain_ckp[-1].restore(best_ckp_path)
  print('Loaded ckp from: ', best_ckp_path)

Loaded ckp from:  Result\UNet4\ckp\best-17
Loaded ckp from:  Result\UNet4_sub1\ckp\best-16
Loaded ckp from:  Result\UNet4_sub2\ckp\best-13
Loaded ckp from:  Result\UNet4_sub3\ckp\best-17
Loaded ckp from:  Result\UNet4_sub4\ckp\best-16
Loaded ckp from:  Result\UNet4_sub5\ckp\best-16
Loaded ckp from:  Result\UNet4_sub6\ckp\best-21
Loaded ckp from:  Result\UNet4_sub7\ckp\best-17
Loaded ckp from:  Result\UNet4_sub8\ckp\best-16


In [9]:
u_opmz = tf.keras.optimizers.Adam(1e-6)
u_moderator = moderator()
ckp = tf.train.Checkpoint(u_optimizer=u_opmz, u_model=u_moderator)

with open(os.path.join(DIR_MODEL, 'UNet4_moderator4', 'ckp', 'best.txt'), 'r') as f:
    best_ckp_path = f.readline()
ckp.restore(best_ckp_path)
print('Loaded ckp from: ', best_ckp_path)

Loaded ckp from:  Result\UNet4_moderator4\ckp\best-8


# Evaluation

In [10]:
dir_img = os.path.join('TrainData', 'Processed')
ls_img_path = glob(os.path.join(dir_img, '*.tif'))
ls_img_path.sort()

dir_geo_tif = os.path.join(DIR_OUTPUT, 'geo', 'tif')
if not os.path.exists(dir_geo_tif): 
  os.makedirs(dir_geo_tif)

dir_geo_plg = os.path.join(DIR_OUTPUT, 'geo', 'plg')
if not os.path.exists(dir_geo_plg): 
  os.makedirs(dir_geo_plg)

In [11]:
def predictTiff_moderator(ls_pretrain, tf_model, cur_img_path): 

  with rasterio.open(cur_img_path) as src:
    cur_data = [src.read(i+1) for i in range(3)]
    meta = src.meta
    
  cur_data = np.stack(cur_data, axis=-1)
  cur_mask = np.all(cur_data == [0,0,0], axis=-1)
  cur_mask = ~cur_mask

  img_xsize = cur_data.shape[0]
  img_ysize = cur_data.shape[1]


  ## ------ predict ------
  res_lab = np.zeros((img_xsize, img_ysize), dtype=np.bool_)
  step_x = IMG_WIDTH // 2
  step_y = IMG_HEIGHT // 2

  # fill boundary 1
  for x in [0, img_xsize-IMG_HEIGHT]: 
    for y in range(0, img_ysize-IMG_WIDTH+1, IMG_WIDTH):
      x1 = x + IMG_HEIGHT
      y1 = y + IMG_WIDTH
      
      if np.sum(cur_mask[x:x1, y:y1]) == 0: 
        continue 
      
      cur_x = tf.image.per_image_standardization(cur_data[x:x1, y:y1])
      cur_x = tf.expand_dims(cur_x, axis=0)
      
      ls_logis = []
      for cur_pretrain in ls_pretrain: 
        cur_logits = cur_pretrain(cur_x, training=False)
        cur_logits = tf.nn.softmax(cur_logits)
        ls_logis.append(cur_logits)
      cur_x = [cur_x] + ls_logis
        
      cur_pred = tf_model(cur_x, training=False)
      cur_pred = tf.nn.softmax(cur_pred)
      cur_pred = tf.cast(tf.argmax(cur_pred[0], axis=-1), tf.bool).numpy()

      res_lab[x:x1, y:y1] = cur_pred

  # fill boundary 2
  for y in [0, img_ysize-IMG_WIDTH]: 
    for x in range(0, img_xsize-IMG_HEIGHT+1, IMG_HEIGHT):
      x1 = x + IMG_HEIGHT
      y1 = y + IMG_WIDTH
      
      if np.sum(cur_mask[x:x1, y:y1]) == 0: 
        continue 
      
      cur_x = tf.image.per_image_standardization(cur_data[x:x1, y:y1])
      cur_x = tf.expand_dims(cur_x, axis=0)
      
      ls_logis = []
      for cur_pretrain in ls_pretrain: 
        cur_logits = cur_pretrain(cur_x, training=False)
        cur_logits = tf.nn.softmax(cur_logits)
        ls_logis.append(cur_logits)
      cur_x = [cur_x] + ls_logis
      
      cur_pred = tf_model(cur_x, training=False)
      cur_pred = tf.nn.softmax(cur_pred)
      cur_pred = tf.cast(tf.argmax(cur_pred[0], axis=-1), tf.bool).numpy()

      res_lab[x:x1, y:y1] = cur_pred

  # fill center
  x_flag = True
  for x in range(0, img_xsize, step_x): 
    ls_x = []
    ls_y_idx = []
    y_flag = True 
    for y in range(0, img_ysize, step_y):
      x1 = x + IMG_HEIGHT
      y1 = y + IMG_WIDTH
      
      if x1 > img_xsize:
        if x_flag: 
          x1 = img_xsize
          x = x1 - IMG_HEIGHT
          x_flag = False
        else: 
          continue
      if y1 > img_ysize:
        if y_flag: 
          y1 = img_ysize
          y = y1 - IMG_WIDTH
          y_flag = False
        else: 
          continue
      
      if np.sum(cur_mask[x:x1, y:y1]) == 0: 
        continue 
      
      ls_y_idx.append(y)       
      cur_x = tf.image.per_image_standardization(cur_data[x:x1, y:y1])
      ls_x.append(tf.expand_dims(cur_x, axis=0))
    
    
    if len(ls_x) > 0: 
      ls_x = tf.concat(ls_x, axis=0)
      ls_x = tf.data.Dataset.from_tensor_slices(ls_x)
      ls_x = ls_x.batch(100)
      
      ls_y = []
      for cur_x_ds in ls_x: 
      
        ls_logis = []
        for cur_pretrain in ls_pretrain: 
          cur_logits = cur_pretrain(cur_x_ds, training=False)
          cur_logits = tf.nn.softmax(cur_logits)
          ls_logis.append(cur_logits)
        cur_x_ds = [cur_x_ds] + ls_logis
        
        cur_pred = tf_model(cur_x_ds, training=False)
        cur_pred = tf.nn.softmax(cur_pred)
        cur_pred = tf.cast(tf.argmax(cur_pred, axis=-1), tf.bool)
        ls_y.append(cur_pred)
      ls_y = tf.concat(ls_y, axis=0).numpy()
          
      for i in range(len(ls_y_idx)):
        y = ls_y_idx[i]
        y1 = y + IMG_WIDTH
        
        res_lab[x+int(step_x/2):x1-int(step_x/2), y+int(step_y/2):y1-int(step_y/2)] = ls_y[i][int(step_x/2):int(step_x/2)+step_x, int(step_y/2):int(step_y/2)+step_y]

    else: 
      continue

  res_lab = np.logical_and(res_lab, cur_mask)
  res_meta = meta.copy()
  res_meta.update({
    'nodata': None,
    'count': 1,
    })

  return res_lab, res_meta


def imgMph(img, kernel_size): 
  kernel = np.ones((kernel_size, kernel_size),np.uint8)
  
  res = img.astype(float)
  res = cv2.morphologyEx(res, cv2.MORPH_CLOSE, kernel)
  res = cv2.morphologyEx(res, cv2.MORPH_OPEN, kernel)
  
  return res.astype(np.uint8)


def img2gdf(np_img, rs_trs, rs_crs, res_name): 
  res_polygon = RSHAPES(np_img, np_img==1, transform=rs_trs)

  ls_geo = []
  for geom, _ in res_polygon:
    ls_geo.append(shapely.geometry.shape(geom))

  gdf = gpd.GeoDataFrame(geometry=ls_geo)
  gdf.crs = rs_crs
  gdf['Area'] = gdf.area
  
  gdf_name = generateOfficialName(res_name.split('_roi')[0])
  gdf_roi = int(res_name.split('_roi')[1][:1]) + 1
  gdf['image'] = gdf_name
  gdf['region_num'] = gdf_roi
  gdf = gdf[['image', 'region_num', 'geometry', 'Area']] # remove Area column before submission
  
  return gdf


def generateOfficialName(str_date):
  NAME_PREFIX = 'Greenland26X_22W_Sentinel2_'
  LS_NAME_SUFFIX = ['_05.tif', '_20.tif', '_25.tif', '_29.tif']
  
  gdf_name = str_date
  if gdf_name == '2019-06-03':
    gdf_name += LS_NAME_SUFFIX[0]
  elif gdf_name == '2019-06-19':
    gdf_name += LS_NAME_SUFFIX[1]
  elif gdf_name == '2019-07-31':
    gdf_name += LS_NAME_SUFFIX[2]
  elif gdf_name == '2019-08-25':
    gdf_name += LS_NAME_SUFFIX[3]
  else: 
    print('Wrong image date detected!')
  gdf_name = NAME_PREFIX + gdf_name
  return gdf_name


def removeOverlay(gdf):
  mask = pd.Series([True] * len(gdf), index=gdf.index)

  for i, poly1 in enumerate(gdf.geometry):
      for j, poly2 in enumerate(gdf.geometry):
          if i != j and poly1.overlaps(poly2):
              mask[i] = False
              break
  
  out = gdf[mask]
  if len(out) != len(gdf): 
    print(f'Found {len(gdf)-len(out)} overlaid polygons. Removed..')
  return out


def computeF1_GPD(gdf_true, gdf_pred):
  tp = 0
  fp = 0
  fn = 0

  threshold = 0.5
  for poly1 in gdf_true.geometry:
      match_found = False
      for poly2 in gdf_pred.geometry:
          overlap_area = poly1.intersection(poly2).area
          
          if overlap_area / poly1.area > threshold:
              match_found = True
              tp += 1
              break
      if not match_found:
          fn += 1

  for poly2 in gdf_pred.geometry:
      if not any(poly1.intersection(poly2).area / poly1.area > threshold for poly1 in gdf_true.geometry):
          fp += 1

  precision = tp / (tp + fp) if tp + fp > 0 else 0
  recall = tp / (tp + fn) if tp + fn > 0 else 0
  f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0
  
  # print(f"F1 Score: {f1_score:.4f}")
  return f1_score


In [23]:
ls_img_path = [
 'TrainData\\Processed\\2019-06-03_roi1.tif',
 'TrainData\\Processed\\2019-06-19_roi2.tif']

In [28]:

ls_output = []
for cur_img_path in ls_img_path: 
  cur_name = os.path.basename(cur_img_path)
  output_name = os.path.join(dir_geo_tif, 'pred_'+cur_name)


  # predic
  if not os.path.exists(output_name):
    res_lab, res_meta = predictTiff_moderator(ls_pretrain, u_moderator, cur_img_path)
    res_lab = res_lab.astype(np.uint8)
    with rasterio.open(output_name, "w", **res_meta) as dest:
      dest.write(res_lab, 1)
  else: 
    with rasterio.open(output_name) as src:
      res_lab = src.read(1)
      res_meta = src.meta
      
      
  # post processing
  # 1. minimal convex hull
  contours, _ = cv2.findContours(res_lab, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  for cnt in contours:
    hull = cv2.convexHull(cnt)
    cv2.drawContours(res_lab, [hull], 0, (255), 2)
  
  # 2. image mophological operation  
  res_post = imgMph(res_lab, 10)
  
  # 3. overlay and area filter
  res_gdf = img2gdf(res_post, res_meta['transform'], res_meta['crs'], cur_name)
  res_gdf = removeOverlay(res_gdf)
  res_gdf = res_gdf[res_gdf.area > 100000]
  
  # 4. stats filter
  with rasterio.open(cur_img_path) as src:
    cur_data = [src.read(i+1) for i in range(3)]
    meta = src.meta
  cur_data = np.stack(cur_data, axis=-1)
  
  ls_flag = []
  for i in range(len(res_gdf)):
    cur_plg = rasterio.features.rasterize(res_gdf.iloc[i:i+1].geometry,
                                          out_shape=(res_meta['height'], res_meta['width']),
                                          transform=res_meta['transform'], dtype='uint16')
    cur_plg_rgb = cur_data[cur_plg==1]
    if np.std(cur_plg_rgb) < 20: 
      # within all training polygons, std of rgb > 20
      cur_flag = False
    else: 
      cur_flag = True
    ls_flag.append(cur_flag)
  res_gdf = res_gdf.loc[ls_flag]

  ls_output.append(res_gdf)

ls_output = pd.concat(ls_output, ignore_index=True)
ls_output = ls_output[['image', 'region_num', 'geometry']]
ls_output.to_file(os.path.join(DIR_OUTPUT, 'lake_poygons_test.gpkg'), driver="GPKG")


100%|██████████| 2/2 [03:42<00:00, 111.27s/it]
